# Embedded CFG Experiment `timer`

Testing the embedded FXE implementation on `timer` example binaries from the nRF5 SDK (v17.0.2)

In [1]:
import os
from copy import deepcopy

from fxe import *

In [2]:
%%time
fxe = FXEngine(
    pd="mmaps/nrf52832.yml",
    path="examples/timer.elf",
    log_stdout=True,
    log_insn=True,
)
fxe.run()

2022-12-26 19:13:07,213 : backup context @ 0x2b1
2022-12-26 19:13:07,213 : backup context @ 0x2b1
2022-12-26 19:13:07,214 : backup context @ 0x2b1
2022-12-26 19:13:07,214 : backup context @ 0x2b1
2022-12-26 19:13:07,214 : backup context @ 0x2b1
2022-12-26 19:13:07,215 : backup context @ 0x2b1
2022-12-26 19:13:07,215 : backup context @ 0x2b1
2022-12-26 19:13:07,215 : backup context @ 0x2b1
2022-12-26 19:13:07,215 : backup context @ 0x2b1
2022-12-26 19:13:07,216 : backup context @ 0x2b1
2022-12-26 19:13:07,216 : backup context @ 0x2b1
2022-12-26 19:13:07,216 : backup context @ 0x2b1
2022-12-26 19:13:07,217 : backup context @ 0x2b1
2022-12-26 19:13:07,217 : backup context @ 0x2b1
2022-12-26 19:13:07,217 : backup context @ 0x2b1
2022-12-26 19:13:07,217 : backup context @ 0x2b1
2022-12-26 19:13:07,218 : backup context @ 0x2b1
2022-12-26 19:13:07,218 : backup context @ 0x2b1
2022-12-26 19:13:07,218 : backup context @ 0x2b1
2022-12-26 19:13:07,218 : backup context @ 0x2b1
2022-12-26 19:13:07,

In [3]:
total_insns = 0
total_size = 0
for block in fxe.cfg.bblocks.values():
    total_size += block.size
    for insn in fxe.cs.disasm(
            fxe.uc.mem_read(block.addr, block.size), block.size):
        total_insns += 1
print(f"Found {len(fxe.cfg.bblocks)} blocks, "
      f"{len(fxe.cfg.edges)} edges, "
      f"{total_insns} insns, "
      f"{hex(total_size)} bytes, "
      "\n")

Found 207 blocks, 229 edges, 861 insns, 0x938 bytes, 



In [4]:
cnt = 0
for branch in fxe.explored:
    if not branch.ret_addr:
        cnt += 1
        print(fxe.fw.disasm_txt[fxe.fw.disasm[branch.addr]['line']])
print(cnt)

 2ea:	e7fe      	b.n	2ea <Default_Handler>
 584:	d1e6      	bne.n	554 <TIMER0_IRQHandler+0x14>
 55c:	b17b      	cbz	r3, 57e <TIMER0_IRQHandler+0x3e>
 56e:	d006      	beq.n	57e <TIMER0_IRQHandler+0x3e>
 2e8:	e7fe      	b.n	2e8 <SysTick_Handler>
 2e6:	e7fe      	b.n	2e6 <PendSV_Handler>
 2e4:	e7fe      	b.n	2e4 <DebugMon_Handler>
 2e2:	e7fe      	b.n	2e2 <SVC_Handler>
 2e0:	e7fe      	b.n	2e0 <UsageFault_Handler>
 2de:	e7fe      	b.n	2de <BusFault_Handler>
 2dc:	e7fe      	b.n	2dc <MemoryManagement_Handler>
 2da:	e7fe      	b.n	2da <HardFault_Handler>
 2d8:	e7fe      	b.n	2d8 <NMI_Handler>
 2c0:	dcfb      	bgt.n	2ba <Reset_Handler+0xa>
 9e2:	e7f9      	b.n	9d8 <memset+0x4>
 406:	d8f0      	bhi.n	3ea <nrfx_timer_init+0x2e>
 50c:	b19e      	cbz	r6, 536 <nrfx_timer_extended_compare+0x5a>
 53c:	e7f3      	b.n	526 <nrfx_timer_extended_compare+0x4a>
 55c:	b17b      	cbz	r3, 57e <TIMER0_IRQHandler+0x3e>
 5d8:	b988      	cbnz	r0, 5fe <main+0x46>
 3b2:	e7fd      	b.n	3b0 <app_error_fault_handler+

NOTE: `wfi` instructions stop unicorn emulation. need to handle that explicitly

In [5]:
disasm_txt = deepcopy(fxe.fw.disasm_txt)
for block in fxe.cfg.bblocks.values():
    insn_addr = block.addr
    for insn in fxe.cs.disasm(
            fxe.uc.mem_read(block.addr, block.size), block.size):
        try:
            lineno = fxe.fw.disasm[insn_addr]['line']
            insn_addr += insn.size
            disasm_txt[lineno] = "{}{}{}{:<100}\u001b[0m".format(
                "\u001b[103m",  # bright yellow background
                "\u001b[30m",   # black foreground
                "\u001b[1m",    # bold font
                disasm_txt[lineno])
        except Exception:
            pass
print('\n'.join(disasm_txt))


examples/timer.elf:     file format elf32-littlearm


Disassembly of section .text:

00000000 <__isr_vector>:
   0:	20010000 	.word	0x20010000
   4:	000002b1 	.word	0x000002b1
   8:	000002d9 	.word	0x000002d9
   c:	000002db 	.word	0x000002db
  10:	000002dd 	.word	0x000002dd
  14:	000002df 	.word	0x000002df
  18:	000002e1 	.word	0x000002e1
	...
  2c:	000002e3 	.word	0x000002e3
  30:	000002e5 	.word	0x000002e5
  34:	00000000 	.word	0x00000000
  38:	000002e7 	.word	0x000002e7
  3c:	000002e9 	.word	0x000002e9
  40:	000002eb 	.word	0x000002eb
  44:	000002eb 	.word	0x000002eb
  48:	000002eb 	.word	0x000002eb
  4c:	000002eb 	.word	0x000002eb
  50:	000002eb 	.word	0x000002eb
  54:	000002eb 	.word	0x000002eb
  58:	000002eb 	.word	0x000002eb
  5c:	000002eb 	.word	0x000002eb
  60:	00000541 	.word	0x00000541
  64:	000002eb 	.word	0x000002eb
  68:	000002eb 	.word	0x000002eb
  6c:	000002eb 	.word	0x000002eb
  70:	000002eb 	.word	0x000002eb
  74:	000002eb 	.word	0x000002eb
  78:	000002eb 	.word	0x00

In [6]:
fxe.cfg.bblocks[0x604].parents

[<efxe.BBlock @ 0x00000602 : 0x00000604>]